# Strong and weak reference

In [1]:
import ctypes
import weakref


def ref_count(address):
    return ctypes.c_long.from_address(address).value

In [2]:
p1 = object()
id_p1 = id(p1)
id_p1

2056429606032

In [3]:
ref_count(id_p1)

1

In [4]:
p2 = p1
id(p2)

2056429606032

In [5]:
ref_count(id_p1)
#! we have two strong reference to that object

2

In [6]:
del p1
ref_count(id_p1)

1

In [7]:
del p2
ref_count(id_p1)

-857893664

# Weak Reference

In [8]:
class Person:
    def __init__(self,name):
        self.name = name
    def __repr__(self):
        return f"Person(name={self.name})"

In [9]:
p1 = Person("A")
id_p1 = id(p1)

In [10]:
ref_count(id_p1)

1

In [11]:
p2 = p1
ref_count(id_p1)

2

In [12]:
weak1 = weakref.ref(p1)

In [13]:
ref_count(id_p1)

2

In [14]:
weak1

<weakref at 0x000001DECCE0FFB0; to 'Person' at 0x000001DECCDFD090>

In [15]:
hex(id_p1)

'0x1deccdfd090'

In [16]:
print(weak1())

Person(name=A)


In [17]:
ref_count(id_p1)

2

In [18]:
weak2 = weakref.ref(p1)

In [19]:
ref_count(id_p1)

2

In [20]:
p1.__weakref__

<weakref at 0x000001DECCE0FFB0; to 'Person' at 0x000001DECCDFD090>

In [21]:
weakref.getweakrefcount(p1)

1

In [22]:
del p1

In [23]:
ref_count(id_p1)

1

In [24]:
weak1

<weakref at 0x000001DECCE0FFB0; to 'Person' at 0x000001DECCDFD090>

In [25]:
del p2

In [26]:
ref_count(id_p1)

0

In [27]:
weak1

<weakref at 0x000001DECCE0FFB0; dead>

Note that not every object in Python supports weak references. Many of the built-in types do not:

In [28]:
l = [1,2,3]
try:
    weakref.ref(l)
except TypeError as ex:
    print(ex)

cannot create weak reference to 'list' object


In [29]:
l = (1,2,3)
try:
    weakref.ref(l)
except TypeError as ex:
    print(ex)

cannot create weak reference to 'tuple' object


In [30]:
l = 10
try:
    weakref.ref(l)
except TypeError as ex:
    print(ex)

cannot create weak reference to 'int' object


In [31]:
l = {}
try:
    weakref.ref(l)
except TypeError as ex:
    print(ex)

cannot create weak reference to 'dict' object


But our custom classes do, and that's what we need here.

For our data descriptors, we want to use the instance objects as keys in our dictionary. But as we saw earlier, storing the object itself as the key can lead to memory leaks. So instead, we are going to store weak references to the object in the dictionary.

We could use our own dictionary, but weakref also provides a specialized dictionary type, that will store a weak reference to the object being used as the key

In [32]:
p1 = Person("A")
d = weakref.WeakKeyDictionary()

In [33]:
ref_count(id(p1))

1

In [34]:
weakref.getweakrefcount(p1)

0

In [35]:
d[p1] = "A"

In [36]:
ref_count(id(p1)),weakref.getweakrefcount(p1)

(1, 1)

In [37]:
hex(id(p1)), list(d.keyrefs())

('0x1decce368d0',
 [<weakref at 0x000001DECCE3C1D0; to 'Person' at 0x000001DECCE368D0>])

In [38]:
del p1

In [39]:
list(d.keyrefs())

[]

It was automatically removed when the object it was pointing to (weakly) was destroyed by the garbage collector!

Now be careful, you can only use keys in the WeakKeyDictionary that Python can create weak references to:

So this will not work:

In [40]:
try:
    d['python'] = 'test'
except TypeError as ex:
    print(ex)

cannot create weak reference to 'str' object


Also, even though we are using a weak reference as a key in the dictionary, the object must still be hashable.

Let's see an example of this:



In [41]:
class Person:
    def __init__(self, name):
        self.name = name

    def __eq__(self, other):
        return isinstance(other, Person) and self.name == other.name

In [42]:
p1 = Person('Guido')
p2 = Person('Guido')


In [43]:
p1 == p2

True

In [44]:
try:
    hash(p1)
except TypeError as ex:
    print(ex)

unhashable type: 'Person'


And so we cannot use it as a key in our WeakKeyDictionary

In [45]:
try:
    d[p1] = 'Guido'
except TypeError as ex:
    print(ex)

unhashable type: 'Person'


So we can certainly use WeakKeyDictionary objects in our data descriptors, but that will only work with hashable objects. In the next lectures we'll look at how to use WeakKeyDictionary as a storage mechanism for our data descriptors, as well as how to deal with the unhashable issue.

In [46]:
class Person:
    def __init__(self, name):
        self.name = name

    def __repr__(self):
        return f"Person(name={self.name})"

    def __eq__(self, other):
        return isinstance(other, Person) and self.name == other.name

    def __hash__(self):
        return hash(self.name)

In [47]:
p1 = Person('Guido')
p2 = Person('Guido')

In [48]:
p1 == p2

True

In [49]:
try:
    print(hash(p1))
except TypeError as ex:
    print(ex)

-267009858766043489


In [50]:
try:
    d[p1] = 'Guido'
except TypeError as ex:
    print(ex)

In [51]:
d.keyrefs()

[<weakref at 0x000001DECCE3EC00; to 'Person' at 0x000001DECCDFD490>]